### Import packages

In [1]:
import pandas as pd
import numpy as np



#%load_ext rpy2.ipython

### Sample 3 individuals from each population:

In [2]:
dataset = 'HGDP'
metadata = pd.read_table("~/GenerationInterval/people/moi/files/metadata.txt")
metadata = metadata[metadata['dat'] == dataset]

In [3]:
pop = 'She'

data = metadata[metadata['pop'] == pop]
samples = data.sample(n=3, random_state=10)
pop_ind = (samples['ind'].tolist())

print(pop_ind)

['HGDP01335', 'HGDP01329', 'HGDP01332']


### Get Neanderthal fragment data for each population

In [4]:
frames = []
for ind in pop_ind:
    human_id = ind
    ind_bed_in = '../../moi/sandbox/dec/{dataset}/{human_id}.new.txt'.format(dataset = dataset, human_id = human_id)
    ind_file = pd.read_csv(ind_bed_in, sep = '\t')
    ind_file["end"] += 1000
    frames.append(ind_file)
    pop_df = pd.concat(frames)
    conditions = [
    (pop_df['ancestry'] == 'Vindija') | (pop_df['ancestry'] == 'AmbigNean') | (pop_df['ancestry'] == 'Altai') | (pop_df['ancestry'] == 'Chagyrskaya'),
    (pop_df['ancestry'] == 'Denisova'),
    (pop_df['ancestry'] == 'nonDAVC'),
    (pop_df['ancestry'] == 'Ambiguous')
    ]
    values = ['Neanderthal', 'Denisova', 'NonDAVC', 'Ambiguous']
    pop_df['group'] = np.select(conditions, values)
    nean_df = pop_df.loc[pop_df['group'] == 'Neanderthal']
    nean_df.to_csv("{pop}.txt".format(pop = pop), index = False, header = True, sep = '\t')


### Make artificial genomes for each population

##### Goal: Pick a random fragment and add it to the artificial genome. Then find all overlapping fragments in df and remove them from df. Then pick a new fragment and add to ag. Find overlaps again and remove them, etc.

In [8]:
%%bash

# Create new empty file for the artificial genome

touch ag.txt


In [9]:
%%bash

# Pick a random fragment and add to the artificial genome

shuf -n 1 Burusho.txt >> ag.txt


In [14]:
%%bash 

head -n 5 Burusho.txt

chrom	start	end	length	mean_prob	snps	admixpopvariants	Altai	Chagyrskaya	Denisova	Vindija	ind	dataset	ancestry	group
chr10	1704000	1717000	14000	0.79346	7	4	0	3	0	4	HGDP00359	HGDP	Vindija	Neanderthal
chr10	1865000	2038000	174000	0.9039	71	29	25	28	2	28	HGDP00359	HGDP	AmbigNean	Neanderthal
chr10	3749000	3824000	76000	0.98651	56	33	32	31	3	32	HGDP00359	HGDP	AmbigNean	Neanderthal
chr10	5564000	5618000	55000	0.95123	27	13	9	13	0	11	HGDP00359	HGDP	Chagyrskaya	Neanderthal


In [18]:
%%bash

head -n 5 Burusho.txt | shuf

chr10	1704000	1717000	14000	0.79346	7	4	0	3	0	4	HGDP00359	HGDP	Vindija	Neanderthal
chrom	start	end	length	mean_prob	snps	admixpopvariants	Altai	Chagyrskaya	Denisova	Vindija	ind	dataset	ancestry	group
chr10	1865000	2038000	174000	0.9039	71	29	25	28	2	28	HGDP00359	HGDP	AmbigNean	Neanderthal
chr10	5564000	5618000	55000	0.95123	27	13	9	13	0	11	HGDP00359	HGDP	Chagyrskaya	Neanderthal
chr10	3749000	3824000	76000	0.98651	56	33	32	31	3	32	HGDP00359	HGDP	AmbigNean	Neanderthal


In [60]:
%%bash

awk '{if(NR > 1){print $1"\t"$2"\t"$3"\t"$12}}' Burusho.txt \
| head -n 5 

chr10	1704000	1717000	HGDP00359
chr10	1865000	2038000	HGDP00359
chr10	3749000	3824000	HGDP00359
chr10	5564000	5618000	HGDP00359
chr10	5661000	5678000	HGDP00359


In [61]:
%%bash

awk '{if(NR > 1){print $1"\t"$2"\t"$3"\t"$12}}' Burusho.txt \
| head -n 5 \
| while read line 
    do 
      echo "I'm doing line" ${line}
      bedtools intersect -v -a <(echo "" | awk '{print "'"${line}"'"}') -b <(awk '{if(NR > 1){print $1"\t"$2"\t"$3"\t"$12}}' Han.txt) 
    done

#<(echo "" | awk '{print "chr10\t1704000\t1717000\tmoi"}')

I'm doing line chr10 1704000 1717000 HGDP00359
chr10	1704000	1717000	HGDP00359
I'm doing line chr10 1865000 2038000 HGDP00359
I'm doing line chr10 3749000 3824000 HGDP00359
I'm doing line chr10 5564000 5618000 HGDP00359
chr10	5564000	5618000	HGDP00359
I'm doing line chr10 5661000 5678000 HGDP00359
chr10	5661000	5678000	HGDP00359


In [70]:
%%bash

cat art_gen.txt 

In [74]:
%%bash

pop="Burusho" 

rm -f ${pop}_art_gen.txt
touch ${pop}_art_gen.txt

awk '{if(NR > 1){print $1"\t"$2"\t"$3"\t"$12}}' ${pop}.txt \
| shuf \
| head -n 5 \
| while read line 
    do 
      bedtools intersect -v -a <(echo "" | awk '{print "'"${line}"'"}') -b ${pop}_art_gen.txt > ${pop}_art_gen_temp.txt
      cat ${pop}_art_gen_temp.txt >> ${pop}_art_gen.txt
      rm ${pop}_art_gen_temp.txt
      
    done

echo ""
cat ${pop}_art_gen.txt
#<(echo "" | awk '{print "chr10\t1704000\t1717000\tmoi"}')


chr10	1865000	2038000	HGDP00359
chr10	3749000	3824000	HGDP00359
chr10	5564000	5618000	HGDP00359
chr10	5661000	5678000	HGDP00359
chr10	1704000	1717000	HGDP00359
chr9	126532000	126647000	HGDP00341
chr7	152720000	152751000	HGDP00359
chr10	90272000	90318000	HGDP00359
chr14	79964000	80041000	HGDP00341
chr15	86930000	86971000	HGDP00359
chr3	123546000	123663000	HGDP00371
chr12	104699000	104711000	HGDP00341
chr6	68786000	68851000	HGDP00359
chr16	55429000	55461000	HGDP00341
chr2	159823000	160009000	HGDP00371


In [30]:
%%bash

# Find all fragments that don't overlap with any fragments in the artificial genome and save them in a new file.

touch tmp.txt

bedtools intersect -a Burusho.txt -b ag.txt -v > tmp.txt


In [11]:
%%bash

# Pick a new random fragment from the non-overlapping fragments and add it to the artificial genome

shuf -n 1 tmp.txt >> ag.txt

In [12]:
%%bash

more ag.txt

::::::::::::::
ag.txt
::::::::::::::
chr3	23692000	23721000	30000	0.84888	9	1	1	1	0	1	HGDP00359	HGDP	AmbigNean	Neanderthal
chr1	233957000	234067000	111000	0.87418	36	6	6	6	3	6	HGDP00371	HGDP	AmbigNean	Neanderthal


##### Goal: Pick a random fragment and add it to the artificial genome. Then pick another random fragment, check if it overlaps with any fragment already in the artificial genome and if not, then add that fragment etc.

In [37]:
ag = pd.DataFrame()
df = pd.read_csv("Burusho.txt", sep = '\t')
first_frag = df.sample(n = 1, axis = 0, random_state = 4)
ag = pd.concat([ag,first_frag])
df = df.drop(first_frag.index.tolist(), axis = 0)

#next_frag = df.sample(n = 1, axis = 0, random_state = 4)
#for i in df.values:
#    for j in ag.values:
#        if df.values[i][1] 
#

while len(df) != 0:
    next_frag = df.sample(n = 1, axis = 0, random_state = 4)
    for row in range(len(ag)):
        if next_frag.iloc[0]['start'] > ag.iloc[row]['start'] & next_frag.iloc[0]['end'] < ag.iloc[row]['end']:
            df = df.drop(next_frag.index.tolist(), axis = 0)
            break
        if next_frag.iloc[0]['start'] < ag.iloc[row]['start'] & next_frag.iloc[0]['end'] > ag.iloc[row]['start']: 
            df = df.drop(next_frag.index.tolist(), axis = 0)
            break
        if next_frag.iloc[0]['start'] < ag.iloc[row]['end'] & next_frag.iloc[0]['end'] > ag.iloc[row]['end']: 
            df = df.drop(next_frag.index.tolist(), axis = 0)
            break
        else:
            ag = pd.concat([ag,next_frag])
            df = df.drop(next_frag.index.tolist(), axis = 0)
            break



      chrom     start       end  length  mean_prob  snps  admixpopvariants  \
1620   chr1  36397000  36560000  164000    0.96128    47                17   
307   chr21  34173000  34219000   47000    0.94734    19                 9   
554    chr7    527000    549000   23000    0.74189     8                 3   
478    chr5   2741000   2804000   64000    0.96867    29                13   
1162   chr5   2753000   2803000   51000    0.96436    26                16   
1601  chr19  33534000  33556000   23000    0.90102    12                10   
487    chr5  36049000  36375000  327000    0.97764   124                55   
476    chr5    174000    574000  401000    0.91008   142                65   
1240   chr7  36166000  36308000  143000    0.94663    45                18   
1107   chr4  33665000  34345000  681000    0.94820   174                69   
1106   chr4  33199000  33585000  387000    0.98642   146                68   
701   chr11  35604000  35657000   54000    0.94786    16        